# Reconhecimento de fala

Quando pensamos em reconhecimento de fala, estamos pensando em modelos de IA capaz de **transcrever** o que alguém está falado em um texto.

## Dataset de trechos de fala

Vamos utilizar agora o Dataset Minds14, que contém trechos de clientes em um atendimento de banco, em múltiplas línguas: https://huggingface.co/datasets/PolyAI/minds14

Esse conjunto de dados não é tão grande (arquivo zipado de 400MB), portanto não vamos baixar dados no modo streaming. Isso facilita a exploração dos dados e agiliza seu carregamento, uma vez baixados. Além disso, vamos usar a construção `"train[:10]"` no argumento `split` para carregar apenas os dados iniciais do split de treino:

In [1]:
from datasets import load_dataset

nome_dataset = "PolyAI/minds14"
lingua_dataset = "pt-PT"
dados = load_dataset(nome_dataset, name=lingua_dataset, split='train[:10]')
dados

C:\Users\Juliano\Desktop\IAs para Imagens e Áudio com Hugging Face\venv\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for PolyAI/minds14 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/minds14
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
    num_rows: 10
})

## Observando os dados

In [2]:
dados[0]

{'path': 'C:\\Users\\Juliano\\.cache\\huggingface\\datasets\\downloads\\extracted\\e65c75428bc4ba62cdc12ed880d1eeac971af0fb966240d308430d8d9e8ea6ca\\pt-PT~CASH_DEPOSIT\\response_4.wav',
 'audio': {'path': 'C:\\Users\\Juliano\\.cache\\huggingface\\datasets\\downloads\\extracted\\e65c75428bc4ba62cdc12ed880d1eeac971af0fb966240d308430d8d9e8ea6ca\\pt-PT~CASH_DEPOSIT\\response_4.wav',
  'array': array([ 0.        ,  0.        ,  0.        , ...,  0.00048828,
         -0.00097656, -0.00048828]),
  'sampling_rate': 8000},
 'transcription': 'Bom dia estou a ligar porque precisava de informações sobre como é que eu posso depositar dinheiro na minha conta',
 'english_transcription': "Good morning I'm calling because I needed information on how I can deposit money into my account",
 'intent_class': 7,
 'lang_id': 11}

In [3]:
import IPython

for linha in dados:
    dados_som = linha['audio']['array']
    taxa_amostragem = linha['audio']['sampling_rate']
    display(IPython.display.Audio(data=dados_som, rate=taxa_amostragem))

# Utilizando um modelo de detecção de voz

Vamos usar o modelo Whisper - o mesmo usado pela OpenAI - a partir do que a própria empresa já liberou no Hugging Face.

Se formos consultar na [página da OpenAI no Hugging Face](https://huggingface.co/openai), veremos diversos modelos Whisper. A principal diferença desses modelos é o seu **tamanho** e sua **capacidade de processar múltiplas linguagens**. Há uma tabela com comparações [neste link do Github da OpenAI](https://github.com/openai/whisper?tab=readme-ov-file#available-models-and-languages). 

Vamos usar um modelo intermediário com suporte para português (além de diversas outras línguas): https://huggingface.co/openai/whisper-medium

## Passando dados ao modelo

Vamos tentar passar dados da forma como já sabemos, para ver se funciona:

In [4]:
from transformers import pipeline

modelo = 'openai/whisper-medium'
reconhecedor_de_fala = pipeline('automatic-speech-recognition', model=modelo)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

C:\Users\Juliano\Desktop\IAs para Imagens e Áudio com Hugging Face\venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Juliano\.cache\huggingface\hub\models--openai--whisper-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [5]:
reconhecedor_de_fala(dados[0]['audio']['array'])

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'text': ' Bă, din aici a obiscut, vreau să vă spună informația așa pe cum este ca o postă de vizitar din aerul din necun.'}

Não parece ter funcionado. Sampling rate está certo?

In [6]:
reconhecedor_de_fala.feature_extractor.sampling_rate

16000

In [6]:
dados[0]['audio']['sampling_rate']

8000

Poderíamos tentar ajustar o sampling rate novamente, mas os modelos Whisper tem mais uma facilidade para nós: se passarmos o dicionário de Audio diretamente, eles ajustam o sampling rate automaticamente:

In [7]:
reconhecedor_de_fala(dados[0]['audio'])

{'text': ' Bom dia, estou a ligar porque precisava de informações sobre como eu posso depositar dinheiro na minha conta.'}

Sucesso! Rode o código abaixo para testar outros exemplos:

In [8]:
idx_audio = 7

audio = dados[idx_audio]['audio']
display(IPython.display.Audio(data=audio['array'], rate=audio['sampling_rate']))

In [9]:
reconhecedor_de_fala(audio)

{'text': ' Posso depositar dinheiro?'}

## Parâmetros do modelo

O modelo Whisper consegue detectar diferentes línguas e também fazer tradução simultânea entre duas línguas.

Nos exemplos que testamos, tudo ocorreu bem - ele adivinhou corretamente o que queríamos fazer. Mas podemos instruir o modelo a fazer uma certa ação passando alguns parâmetros adicionais:

In [10]:
modelo = 'openai/whisper-medium'

reconhecedor_de_fala = pipeline(
    'automatic-speech-recognition', 
    model=modelo, 
    generate_kwargs={"task": "transcribe", "language": "portuguese"},
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
idx_audio = 7

audio = dados[idx_audio]['audio']
reconhecedor_de_fala(audio)

{'text': ' Posso depositar dinheiro?'}

In [12]:
reconhecedor_de_fala = pipeline(
    'automatic-speech-recognition',
    model=modelo,
    generate_kwargs={"task": "translate", "language": "portuguese"},
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
idx_audio = 7

audio = dados[idx_audio]['audio']
reconhecedor_de_fala(audio)

{'text': ' Can I deposit money?'}

# Testando com nossa própria voz

Vamos usar o microfone para testar a transcrição e tradução:

In [14]:
modelo = 'openai/whisper-medium'

reconhecedor_de_fala = pipeline(
    'automatic-speech-recognition',
    model=modelo,
    generate_kwargs={"task": "transcribe", "language": "portuguese"},
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
import sounddevice as sd

duracao = 10
taxa_amostragem = 16000
tamanho_vetor = int(duracao * taxa_amostragem)

gravacao = sd.rec(tamanho_vetor, samplerate=taxa_amostragem, channels=1)
sd.wait()

gravacao

array([[-0.00018311],
       [-0.00012207],
       [-0.00054932],
       ...,
       [ 0.00100708],
       [ 0.0015564 ],
       [ 0.00042725]], dtype=float32)

In [16]:
reconhecedor_de_fala({'raw': gravacao.ravel(), 'sampling_rate': taxa_amostragem})

{'text': ' A pequena ilha de Ikaria, no leste do mar Egeu, é considerada uma das cinco zonas azuis, os locais do mundo onde as pessoas vivem por mais tempo.'}

In [17]:
modelo = 'openai/whisper-medium'

reconhecedor_de_fala = pipeline(
    'automatic-speech-recognition',
    model=modelo,
    generate_kwargs={"task": "translate", "language": "portuguese"},
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
reconhecedor_de_fala({'raw': gravacao.ravel(), 'sampling_rate': taxa_amostragem})

{'text': ' The small island of Ikaria, east of the Aegean Sea, is considered one of the five blue zones, the places in the world where people live for the longest time.'}

Sucesso!